In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

############## datafarme 보여주는 범위 설정 ###################
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 200
pd.options.display.float_format = '{:.4f}'.format

In [ ]:
path="C:/Users/user/Documents/GitHub/taas/"

load_path = path + 'pm_data/'  # 데이터 불러오는 경로
save_path = path + 'preprocessing/'  # 전처리 이후 데이터 저장하는 경로

In [ ]:
# raw_data
df = pd.read_csv(load_path + '2017_2020_data.csv', encoding = "EUC-KR")
# 변수 정리 파일(name, type, 제거 여부 등)
cate=pd.read_csv(load_path + 'catego3.csv',encoding = "EUC-KR")

In [ ]:
# 원동기장치자전거 제외
df_ljc = df.copy()
df_pmbycmoto=df_ljc[df_ljc['당사자종별가해자_대분류']!='원동기장치자전거']
df_pmbycmoto.reset_index(inplace=True, drop=True)

In [ ]:
# 분석에 필요한 변수만 filter
dt=df_pmbycmoto.drop([x for x in cate[cate['method']=='delete'].loc[:,'column']],axis=1,inplace=False)
dt.drop(['index'],axis=1,inplace=True)
dt.drop(['당사자종별가해자_대분류'],axis=1,inplace=True)
dt.reset_index(inplace=True, drop=True)

In [ ]:
# y : 종속변수, x : 독립변수
dt_y=dt['Class']
dt_X=dt.drop(['Class'],axis=1,inplace=False)

In [ ]:
# train, test 8 : 2로 나누기
train_X, test_X, train_y, test_y = train_test_split(dt_X,dt_y,test_size=0.2, random_state=0,stratify = dt_y)

train_X.reset_index(inplace=True,drop=True)
test_X.reset_index(inplace=True,drop=True)
train_y.reset_index(inplace=True,drop=True)
test_y.reset_index(inplace=True,drop=True)

In [ ]:
cate['method'].unique()

In [ ]:
# 지정한 변수 타입별로 분류
dummy = [x for x in cate[cate['method']=='dummy'].iloc[:,0]]
binary = [x for x in cate[cate['method']=='binary'].iloc[:,0]]
ordinal = [x for x in cate[cate['method']=='ordinal'].iloc[:,0]]
leave = [x for x in cate[cate['method']=='LeaveOneOut'].iloc[:,0]]
binning = [x for x in cate[cate['method']=='binning'].iloc[:,0]]
numeric = [x for x in cate[cate['method']=='numeric'].iloc[:,0]]

# 저장 def

In [ ]:
def save_df(data):
    global save_pre
    if save_pre.empty:
        save_pre=data
    else:
        save_pre=pd.concat([save_pre,data],axis=1)

In [ ]:
def save_df_test(data):
    global save_pre_t
    if save_pre_t.empty:
        save_pre_t=data
    else:
        save_pre_t=pd.concat([save_pre_t,data],axis=1)

In [ ]:
save_pre=pd.DataFrame()
save_pre_t=pd.DataFrame()

# ----------------------------------------------------------------------------------------------------------

# LeaveOneOut 변수 효과를 살펴보기 위한 시각화 코드

#### 준비 데이터 : (사고유형별) 전처리 완료한 df, leaveoneout 하기 전 기존 변수를 가지고 있는 df

In [ ]:
# 한글 깨짐 지정
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

sns.set(rc = {'figure.figsize':(13,6)})
sns.set_style('whitegrid')
plt.rcParams['font.family'] = 'NanumGothic'


In [ ]:
def loo_pca_plot(df, temp):
    # loo 변수들 체크하기 위해 이전 변수들과 합쳐주기(PCA 진행할 때 변수들 따로 빼놔야할 듯)
    LOO_Check=pd.concat([df.loc[:,'acci_M':'cross_form'],
                         temp.loc[:,'사고유형_중분류':'교차로형태']],axis=1)
    # 변수 생성을 위해 class 붙여주고,
    LOO_Check['class']=temp['class']
    # loo(PCA) 수치와 일치하는 기존 변수명 매칭
    LOO_Check_df=pd.DataFrame()
    columns_LOO = ['사고유형_중분류', '법규위반가해자', '행동유형가해자', '도로형태', '도로선형_중분류', '도로선형', '교차로형태']
    columns_LOO_eng=['acci_M','law_viol','at_activ','road_form','road_straight_M','road_straight','cross_form']
    
    for i in range(len(columns_LOO)):
        LOO_Check_df=pd.concat([LOO_Check_df,LOO_Check.loc[:,columns_LOO_eng[i]]],axis=1)
        col_temp=pd.DataFrame()
        # _class 변수에 기존 class 에 변수 값을 넣기
        col_temp[columns_LOO_eng[i]+'_class'] =LOO_Check[[columns_LOO[i],'class']].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
        LOO_Check_df=pd.concat([LOO_Check_df,col_temp],axis=1)

    # loo 변수별 수치로 sort 를 한 후, class 이름과 loo 수치를 매칭한 barplot 생성
    for i in range(len(columns_LOO_eng)):
        temp_df=LOO_Check_df.iloc[:,i*2:i*2+2].drop_duplicates()
        a=temp_df.sort_values(by=columns_LOO_eng[i])
        sns.barplot(
        data= a,
        x= columns_LOO_eng[i],
        y= columns_LOO_eng[i]+"_class",
        )

        pd.crosstab(temp['class'], temp[columns_LOO[i]], normalize = 'index').plot(kind = 'bar', rot = 45) 
        plt.legend(bbox_to_anchor = (1.05, 0.6))
        plt.title(columns_LOO[i], size = 18)
        plt.show()
        print("---------------------------------------------------------------------------------------------------------------------------")

In [ ]:
loo_pca_plot(df_vv_train_loo_minmax, temp_df_vv_train) # 전처리된 df, 기존 변수 있는 df : 

# LeaveOneOut 변수 상호작용 영향력 확인(3d plot)

#### 필요 데이터 : (사고유형별) loo 주성분 전에 기존 변수를 붙여놓았던 df or loo 변수 순서형으로 변환한 df

In [ ]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm

plt.rcParams['font.family'] = 'NanumGothic'

In [ ]:
def barplot_3d(df, x, y): # 원하는 df, 상호작용 확인할 변수 x, y
    df_byc = df[df['class'] == 0]
    df_pm = df[df['class'] == 1]
    df_motor = df[df['class'] == 2]
    
    rs = pd.crosstab(df_byc[y], df_byc[x], normalize = 'all')
    result = np.array(rs, dtype = 'float')
    
    fig = plt.figure(figsize=(10,8), dpi = 100)
    ax1=fig.add_subplot(111, projection='3d')
    
    xlabels = np.array(rs.columns) 
    xpos = np.arange(xlabels.shape[0])
    
    ylabels = np.array(rs.index, dtype = 'object')
    ypos = np.arange(ylabels.shape[0])
    
    xposM, yposM = np.meshgrid(xpos, ypos, copy=False)
    
    zpos=result
    zpos = zpos.ravel()
    
    dx=0.5 
    dy=0.5
    dz=zpos
    
    ax1.w_xaxis.set_ticks(xpos + dx/2.)
    ax1.w_xaxis.set_ticklabels(xlabels, size = 7)
    
    ax1.w_yaxis.set_ticks(ypos + dy/2.)
    ax1.w_yaxis.set_ticklabels(ylabels, size = 7)
    
    values = np.linspace(0.2, 1., xposM.ravel().shape[0])
    colors = cm.rainbow(values)
    ax1.bar3d(xposM.ravel(), yposM.ravel(), dz*0, dx, dy, dz, color=colors)

    plt.title(x + " X " + y + " (bicycle)")
    plt.show()

    rs = pd.crosstab(df_pm[y], df_pm[x], normalize = 'all')
    result = np.array(rs, dtype = 'float')
    
    fig = plt.figure(figsize=(10,8), dpi = 100)
    ax1=fig.add_subplot(111, projection='3d')
    
    xlabels = np.array(rs.columns) 
    xpos = np.arange(xlabels.shape[0])
    
    ylabels = np.array(rs.index, dtype = 'object')
    ypos = np.arange(ylabels.shape[0])
    
    xposM, yposM = np.meshgrid(xpos, ypos, copy=False)
    
    zpos=result
    zpos = zpos.ravel()
    
    dx=0.5 
    dy=0.5
    dz=zpos
    
    ax1.w_xaxis.set_ticks(xpos + dx/2.)
    ax1.w_xaxis.set_ticklabels(xlabels, size = 7)
    
    ax1.w_yaxis.set_ticks(ypos + dy/2.)
    ax1.w_yaxis.set_ticklabels(ylabels, size = 7)
    
    values = np.linspace(0.2, 1., xposM.ravel().shape[0])
    colors = cm.rainbow(values)
    ax1.bar3d(xposM.ravel(), yposM.ravel(), dz*0, dx, dy, dz, color=colors)

    plt.title(x + " X " + y + " (PM)")
    plt.show()
    

    rs = pd.crosstab(df_motor[y], df_motor[x], normalize = 'all')
    result = np.array(rs, dtype = 'float')
    
    fig = plt.figure(figsize=(10,8), dpi = 100)
    ax1=fig.add_subplot(111, projection='3d')
    
    xlabels = np.array(rs.columns) 
    xpos = np.arange(xlabels.shape[0])
    
    ylabels = np.array(rs.index, dtype = 'object')
    ypos = np.arange(ylabels.shape[0])
    
    xposM, yposM = np.meshgrid(xpos, ypos, copy=False)
    
    zpos=result
    zpos = zpos.ravel()
    
    dx=0.5 
    dy=0.5
    dz=zpos
    
    ax1.w_xaxis.set_ticks(xpos + dx/2.)
    ax1.w_xaxis.set_ticklabels(xlabels, size = 7)
    
    ax1.w_yaxis.set_ticks(ypos + dy/2.)
    ax1.w_yaxis.set_ticklabels(ylabels, size = 7)
    
    values = np.linspace(0.2, 1., xposM.ravel().shape[0])
    colors = cm.rainbow(values)
    ax1.bar3d(xposM.ravel(), yposM.ravel(), dz*0, dx, dy, dz, color=colors)

    plt.title(x + " X " + y + " (motor)")
    plt.show()

In [ ]:
barplot_3d(temp_df_vv_train, "tree_n", "행동유형가해자")

# ------------------------------------------------------------------------------------------------------

## 상호작용 그래프 그리기 

In [ ]:
import pickle

with open('C:/Users/user/Documents/GitHub/taas/inter_plot_fin/inter_plot_fin', 'rb') as file:
    proba_check_inter = pickle.load(file)

In [ ]:
namelist = list(proba_check_inter.keys())

In [ ]:
# 상호작용 3개 다 잘 있나 확인 
for i in range(len(namelist)):
    name = namelist[i]
    df = proba_check_inter[name]
    if len(df.columns) == 2:
        print(name)
        print(df.columns)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

for i in range(len(namelist)):
    name = namelist[i]
    print(name)
    df = proba_check_inter[name]
    sns.catplot(df.columns[0], df.columns[2], hue = df.columns[1],
                kind = "point", data = df)
    plt.title(name)
    plt.show() 
    
    df.groupby(df.columns[0])[[df.columns[2]]].mean().plot(color = 'purple', legend = False)
    plt.hlines(df.groupby(df.columns[1])[[df.columns[2]]].mean(),0,1,color = ["royalblue", "orangered", "green"])
    plt.title(df.columns[0] + " X " +  df.columns[1])
    plt.show()

# 개요 부분 시각화 

## 1. 서울 pm 가해자 / 피해자 사고 빈도수 (barplot)

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv("C:/Users/user/Documents/GitHub/taas/pm_data/가해자data_2016_2020.csv", encoding = "EUC-KR")
data1 = pd.read_csv("C:/Users/user/Documents/GitHub/taas/pm_data/피해자data_2016_2020.csv", encoding = "EUC-KR")

In [ ]:
# 서울 pm 가해자 ( 피해자 pm 제외 )
pm = data[data['당사자종별가해자'] == "개인형이동수단(PM)"]
pm = pm[pm['차량용도피해자_대분류'] != "개인형이동수단(PM)"]
pm = pm.iloc[:, : -10]
pm['attack'] = 1

In [ ]:
# 서울 pm 피해자 데이터 ( 가해자 pm 데이터 제거 )
pm1 = data1[data1['당사자종별가해자'] != "개인형이동수단(PM)"]
pm1 = pm1[pm1['발생지_시도'] == '서울']
pm1['attack'] = 0

In [ ]:
pm = pd.concat([pm, pm1], axis = 0)
pm = pm.reset_index(drop = True)

## 연도별 pm 사고 발생 빈도

In [ ]:
pm['발생일시'] = pm['발생일시'].astype('str')
pm['year'] = pm['발생일시'].str.slice(start=0,stop=4)
pm['year'] = pd.Categorical(pm['year'], ['2017','2018','2019','2020'], ordered = False)

In [ ]:
pm[pm['year'] == 2016]

In [ ]:
# 한글 깨짐 지정
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

sns.set(rc = {'figure.figsize':(13,6)})
sns.set_style('white')
plt.rcParams['font.family'] = 'NanumGothic'

barplot = sns.countplot("year", hue = 'attack',palette = "Set2", data = pm)

for p in barplot.patches:
    barplot.annotate(p.get_height(), 
                        (p.get_x() + p.get_width() / 2.0, 
                         p.get_height()), 
                        ha = 'center', 
                        va = 'center', 
                        xytext = (0, 5),
                        textcoords = 'offset points')
plt.title("연간 서울 지역 PM 사고 피해자/가해자 빈도",  size = 20)
plt.xlabel("")
plt.ylabel("")

# EDA

In [ ]:
# 한글 깨짐 지정
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

dt=df_pmbycmoto
dt['Class']

In [ ]:
# 처리 필요한 변수들 처리

dt = dt[dt['연령가해자'] != '불명']
dt['가해자연령'] = dt['연령가해자'].str[:-1].astype('int64')
c_map = {0 : '자전거', 1 : 'PM', 2 : '이륜차'}
dt['차종'] = dt['Class'].map(c_map)
dt['차종'] = pd.Categorical(dt['차종'], categories=['자전거', 'PM', '이륜차'], ordered = False)
dt['사고유형_대분류'] = pd.Categorical(dt['사고유형_대분류'], categories=['차대차', '차대사람', '차량단독'], ordered = False)

In [ ]:
sns.set(rc = {'figure.figsize':(12,8)})
sns.set_style('white')
plt.rcParams['font.family'] = 'NanumGothic'


pp = dt['차종'].value_counts()
#define Seaborn color palette to use
colors = sns.color_palette('Set2')[0: 2]
explode = [0, 0.15]
wedgeprops={'width': 0.7, 'edgecolor': 'w', 'linewidth': 5}
pp.plot.pie(autopct='%.2f%%', colors = colors, shadow = True, explode = explode,
       textprops={'size': 'x-large', 'color' : 'black'}) #  도넛모양 :  , wedgeprops=wedgeprops)
plt.xlabel(" ")

In [ ]:
pp = dt['성별가해자_1'].value_counts()
pp

#define Seaborn color palette to use
colors = sns.color_palette('Set2')[0: 4]
explode = [0, 0.1,0.2, 0.3]
wedgeprops={'width': 0.7, 'edgecolor': 'w', 'linewidth': 5}
pp.plot.pie(autopct='%.2f%%', colors = colors, shadow = True, explode = explode,
       textprops={'size': 'x-large', 'color' : 'black'}) #  도넛모양 :  , wedgeprops=wedgeprops)
plt.xlabel(" ")

In [ ]:
# 차종 별 성별 가해자
ct = pd.DataFrame(pd.crosstab(dt['차종'], dt['성별가해자'], normalize = 'index'))
stacked = ct.stack().reset_index().rename(columns={0:'value'})
sns.barplot(x= '차종', y = 'value', hue='성별가해자', data = stacked, palette = 'Set2')

In [ ]:
# 차종 별 성별 가해자
dt['성별피해자'] = pd.Categorical(dt['성별가해자_1'], categories = ['남', '여', '없음', '기타불명'])
ct = pd.DataFrame(pd.crosstab(dt['차종'], dt['성별피해자'], normalize = 'index'))
stacked = ct.stack().reset_index().rename(columns={0:'value'})
sns.barplot(x= '차종', y = 'value', hue='성별피해자', data = stacked, palette = 'Set2')

In [ ]:
# 차종별 사용 나이대

sns.violinplot(x= '차종', y = '가해자연령', data = dt, palette = 'Set2')
# plt.title("차종 별 가해자 연령", size = 20)

In [ ]:
#define Seaborn color palette to use
colors = sns.color_palette('Set2')[0:5]

#create pie chart
plt.pie(data, labels = labels, colors = colors, autopct='%.0f%%')
plt.show()

In [ ]:
# 차종별 사고유형 비율 (차대차, 차대사람, 차량단독)
ct = pd.DataFrame(pd.crosstab(dt['차종'], dt['사고유형_대분류'], normalize = 'index'))
stacked = ct.stack().reset_index().rename(columns={0:'value'})
sns.barplot(x= '차종', y = 'value', hue='사고유형_대분류', data = stacked, palette = 'Set2')

In [ ]:
# 차종별 직진/교차로 사고 비율 (차대차, 차대사람, 차량단독)
ct = pd.DataFrame(pd.crosstab(dt['차종'], dt['사고유형_대분류'], normalize = 'index'))
stacked = ct.stack().reset_index().rename(columns={0:'value'})
sns.barplot(x= '차종', y = 'value', hue='사고유형_대분류', data = stacked, palette = 'Set2')

In [ ]:
# 차종별 직진/교차로 사고 비율 (차대차, 차대사람, 차량단독)
ct = pd.DataFrame(pd.crosstab(dt['차종'], dt['교차로형태_대분류'], normalize = 'index'))
stacked = ct.stack().reset_index().rename(columns={0:'value'})
dtsns.barplot(x= '차종', y = 'value', hue='교차로형태_대분류', data = stacked, palette = 'Set2')

In [ ]:
dt['차종'].value_counts()

In [ ]:
bicycle = dt[dt['차종'] == "자전거"]
pm = dt[dt['차종'] == "PM"]
motor = dt[dt['차종'] == "이륜차"]

sns.kdeplot("road_num", shade = True, data = bicycle)
sns.kdeplot("road_num", shade = True,data = pm)
sns.kdeplot("road_num", shade = True,data = motor)
plt.legend(['자전거', 'PM', '이륜차'])

In [ ]:
sns.catplot("성별가해자", "가해자연령", hue = "차종" ,
            kind = "point", data = dt, palette = "Set2")
plt.show() 

In [ ]:
sns.catplot("성별피해자", "피해자연령", hue = "차종" ,
            kind = "point", data = dt, palette = "Set2")
plt.show() 